# **文本分类 API（预览版）**

## **什么是文本分类？**

顾名思义，文本分类是将标签或类别应用于文本的过程。

常见用例包括：

- 将电子邮件分类为垃圾邮件或非垃圾邮件
- 根据客户评论将情绪分析为正面或负面
- 应用标签来支持工单

## **用机器学习解决文本分类**

分类是机器学习中的一个常见问题。您可以使用多种算法来训练分类模型。文本分类是分类的一个子类别，专门处理原始文本。文本提出了有趣的挑战，因为您必须考虑文本出现的上下文和语义。因此，编码含义和上下文可能很困难。近年来，深度学习模型已成为解决自然语言问题的一种有前途的技术。更具体地说，一种称为转换器的神经网络已成为解决自然语言问题（如文本分类、翻译、摘要和问答）的主要方式。

论文中介绍了Transformers [Attention is all you need](https://arxiv.org/abs/1706.03762)。一些用于自然语言任务的流行转换器架构包括：

- 来自 Transformers (BERT) 的双向编码器表示
- 稳健优化的 BERT 预训练方法 (RoBERTa)
- 生成式预训练 Transformer 2 (GPT-2)
- 生成式预训练 Transformer 3 (GPT-3)

在高层次上，转换器是由编码和解码层组成的模型架构。编码器将原始文本作为输入，并将输入映射到数字表示（包括上下文）以生成特征。解码器使用来自编码器的信息来产生输出，例如在文本分类的情况下的类别或标签。使这些层如此特别的是注意力的概念。注意力是根据上下文相对于序列中其他输入的重要性关注输入的特定部分的想法。例如，假设我正在根据标题对新闻文章进行分类。并非标题中的所有词都是相关的。在“汽车销售处于历史最高水平”这样的标题中，“销售”这样的词可能会受到更多关注，并导致将文章标记为商业或金融。

像大多数神经网络一样，从头开始训练转换器可能会很昂贵，因为它们需要大量数据和计算。但是，您不必总是从头开始训练。使用称为微调的技术，您可以采用预先训练的模型并使用您自己的数据重新训练特定于您的域或问题的层。这为您提供了一个更适合解决您的问题的模型，而无需从头开始训练整个模型的过程。

## **文本分类 API（预览版）**

现在您已经大致了解了如何使用深度学习解决文本分类问题，让我们来看看我们如何将其中的许多技术整合到文本分类 API 中。

文本分类 API 由 [TorchSharp](https://github.com/dotnet/TorchSharp) 提供支持。 TorchSharp 是一个 .NET 库，提供对 libtorch 的访问，该库为 PyTorch 提供支持。 TorchSharp 包含用于在 .NET 中从头开始训练神经网络的构建块。然而，TorchSharp 组件是底层级别的的，从头开始构建神经网络具有陡峭的学习曲线。在 ML.NET 中，我们已将部分复杂性抽象到场景级别。

## 安装包

要使用文本分类 API，您必须安装以下软件包

- [`Microsoft.ML`](https://www.nuget.org/packages/Microsoft.ML/)
- [`Microsoft.ML.TorchSharp`](https://www.nuget.org/packages/Microsoft.ML.TorchSharp/)
- [`TorchSharp-cpu`](https://www.nuget.org/packages/TorchSharp-cpu/) 如果您使用的是 CPU 或 [`TorchSharp-cuda-windows`](https://www.nuget.org/packages/TorchSharp-cuda-windows/) / [`TorchSharp-cuda-linux`](https://www.nuget.org/packages/TorchSharp-cuda-linux/) 如果您使用的是 GPU .

要启用 GPU 支持，您还必须安装 CUDA 依赖项。 有关详细信息，请参阅 [GPU 支持指南](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/install-gpu-model-builder#install-dependencies)。

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"

#r "nuget:Microsoft.ML,2.0.0-preview.22324.1"
#r "nuget:Microsoft.ML.TorchSharp,0.20.0-preview.22324.1"
#r "nuget:TorchSharp-cpu,0.96.7"
#r "nuget:Microsoft.Data.Analysis,0.20.0-preview.22324.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22324.1 Microsoft.ML, 2.0.0-preview.22324.1 Microsoft.ML.TorchSharp, 0.20.0-preview.22324.1 TorchSharp-cpu, 0.96.7

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

## **添加 using 语句**

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.ML.TorchSharp;

## **初始化 MLContext**

所有 ML.NET 操作都从 MLContext 类开始。 初始化 mlContext 会创建一个新的 ML.NET 环境，该环境可以在模型创建工作流对象之间共享。 它在概念上类似于实体框架中的 DBContext。

In [1]:
var mlContext = new MLContext();

### **下载或查找数据**
以下代码尝试在几个已知位置找到数据文件，或者它将从已知的 GitHub 位置下载它。

In [1]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

In [1]:
var yelp_reviews = EnsureDataSetDownloaded("yelp_labelled.txt");
var columnNames = new [] {"Text", "Sentiment"};
var df = DataFrame.LoadCsvFromString(yelp_reviews, separator:'\t',header:false, columnNames:columnNames);

yelp_labelled.txt found here: C:\dev\csharp-notebooks\machine-learning\data\yelp_labelled.txt


加载数据后，使用 `Head` 方法预览前三行。

In [1]:
df.Head(3)

index,Text,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0


> 本教程的数据集使用来自 Kotzias 等人的“使用深度特征从组到单个标签”的数据集。 KDD 2015，并托管在 UCI 机器学习存储库 - Dua, D. 和 Karra Taniskidou, E. (2017)。 [UCI 机器学习存储库](http://archive.ics.uci.edu/ml)。 加利福尼亚州欧文：加利福尼亚大学信息与计算机科学学院。

数据集包含两列：

- **文本:** 来自 Yelp 的原始评论文本
- **情绪:** 表示评论情绪的二进制值。 0 为负数，1 为正数。

## **将数据拆分为训练集和测试集**

原始数据集分为两个子集：训练和测试。 训练集是您用来学习数据模式的工具。 测试集用于使用分类任务的评估指标来评估模型的性能。

在这种情况下，80% 的数据用于由“testFraction”参数定义的训练。 剩下的 20% 用于评估和测试。

In [1]:
var trainTestSplit = mlContext.Data.TrainTestSplit(df, testFraction:0.2);

## **定义训练管道**

文本分类 API 是多类分类目录的一部分。 要使用它，请将 `TextClassification` 训练器添加到您的管道中。

In [1]:
var pipeline =
		mlContext.Transforms.Conversion.MapValueToKey("Label","Sentiment")
			.Append(mlContext.MulticlassClassification.Trainers.TextClassification(sentence1ColumnName: "Text"))
			.Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

## **训练模型**

使用训练数据集使用 `Fit` 方法训练您的模型

In [1]:
var model = pipeline.Fit(trainTestSplit.TrainSet);

## **使用模型进行预测**

通过调用“Transform”方法，使用您的模型对测试数据集进行预测。

In [1]:
var predictionIDV = model.Transform(trainTestSplit.TestSet);

调用 `Transform` 的结果是带有您的预测值的 `IDataView`。 为了更容易查看您的预测，请将 `IDataView` 转换为 `IDataFrame` 。 在这种情况下，我感兴趣的唯一列是 Text、Sentiment（实际值）和 PredictedLabel（预测值）。

In [1]:
var columnsToSelect = new [] {"Text", "Sentiment", "PredictedLabel"};

var predictions = predictionIDV.ToDataFrame(columnsToSelect);

使用 `Tail` 方法预览预测 `DataFrame` 中的最后三行。

In [1]:
predictions.Tail(3)

index,Text,Sentiment,PredictedLabel
0,"Oh this is such a thing of beauty, this restaurant.",1,0
1,"A greasy, unhealthy meal.",0,1
2,"The best place in Vegas for breakfast (just check out a Sat, or Sun.",1,1


## **评估模型**

您可以使用多种指标来评估模型的性能。 使用 [Evaluate](https://docs.microsoft.com/dotnet/api/microsoft.ml.multiclassclassificationcatalog.evaluate?view=ml-dotnet) 方法使用预测“IDataView”计算模型的评估指标。

In [1]:
var evaluationMetrics = 
	mlContext
		.MulticlassClassification
		.Evaluate(predictionIDV);

然后，显示评估指标。 有关多类分类评估指标的更多信息，请参阅 [ML.NET 评估指标指南](https://docs.microsoft.com/dotnet/machine-learning/resources/metrics#evaluation-metrics-for-multi-class- 分类）。

In [1]:
evaluationMetrics

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,TopKAccuracyForAllK,PerClassLogLoss,ConfusionMatrix
10.53512863047496,-14.199291365827746,0.6737016700983757,0.6737967914438503,0,0,<null>,"[ 10.940300196581468, 10.134267400178105 ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.6777777777777778, 0.6701030927835051 ], PerClassRecall: [ 0.6559139784946236, 0.6914893617021277 ], Counts: [ [ 61, 32 ], [ 29, 65 ] ], NumberOfClasses: 2 }"
